In [1]:
import stanza
from e2edutch.predict import Predictor
import e2edutch.download
import io
import e2edutch.conll

Instructions for updating:
non-resource variables are not supported in the long term


/home/dafne/anaconda3/envs/filtbubtest/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Download data, if needed, for stanza and e2edutch

In [2]:
stanza.download('nl')

2021-01-08 14:58:33 INFO: Downloading default packages for language: nl (Dutch)...
2021-01-08 14:58:34 INFO: File exists: /home/dafne/stanza_resources/nl/default.zip.
2021-01-08 14:58:40 INFO: Finished downloading models and saved to /home/dafne/stanza_resources.


In [5]:
e2edutch.download.download_data()

Here's the piece of text we want to analyze:

In [29]:
txt = 'Dit is een stukje tekst. Deze tekst dient als voorbeeld.'

First run stanza

In [30]:
nlp = stanza.Pipeline('nl', processors='tokenize,lemma,pos,depparse')

2021-01-08 15:21:47 INFO: Loading these models for language: nl (Dutch):
| Processor | Package |
-----------------------
| tokenize  | alpino  |
| pos       | alpino  |
| lemma     | alpino  |
| depparse  | alpino  |

INFO:stanza:Loading these models for language: nl (Dutch):
| Processor | Package |
-----------------------
| tokenize  | alpino  |
| pos       | alpino  |
| lemma     | alpino  |
| depparse  | alpino  |

2021-01-08 15:21:47 INFO: Use device: cpu
INFO:stanza:Use device: cpu
2021-01-08 15:21:47 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2021-01-08 15:21:47 INFO: Loading: pos
INFO:stanza:Loading: pos
2021-01-08 15:21:48 INFO: Loading: lemma
INFO:stanza:Loading: lemma
2021-01-08 15:21:48 INFO: Loading: depparse
INFO:stanza:Loading: depparse
2021-01-08 15:21:50 INFO: Done loading processors!
INFO:stanza:Done loading processors!


In [31]:
stanza_doc = nlp(txt)

In [32]:
stanza_doc

[
  [
    {
      "id": 1,
      "text": "Dit",
      "lemma": "dit",
      "upos": "PRON",
      "xpos": "VNW|aanw|pron|stan|vol|3o|ev",
      "feats": "Person=3|PronType=Dem",
      "head": 4,
      "deprel": "nsubj",
      "misc": "start_char=0|end_char=3"
    },
    {
      "id": 2,
      "text": "is",
      "lemma": "zijn",
      "upos": "AUX",
      "xpos": "WW|pv|tgw|ev",
      "feats": "Number=Sing|Tense=Pres|VerbForm=Fin",
      "head": 4,
      "deprel": "cop",
      "misc": "start_char=4|end_char=6"
    },
    {
      "id": 3,
      "text": "een",
      "lemma": "een",
      "upos": "DET",
      "xpos": "LID|onbep|stan|agr",
      "feats": "Definite=Ind",
      "head": 4,
      "deprel": "det",
      "misc": "start_char=7|end_char=10"
    },
    {
      "id": 4,
      "text": "stukje",
      "lemma": "stuk",
      "upos": "NOUN",
      "xpos": "N|soort|ev|dim|onz|stan",
      "feats": "Gender=Neut|Number=Sing",
      "head": 0,
      "deprel": "root",
      "misc": "start_ch

To use the output of stanza for e2edutch, we only need the output of the tokenization:

In [33]:
e2e_doc = {'doc_key': 'example',
          'sentences': [[token['text'] for token in sentence]
                       for sentence in stanza_doc.to_dict()]}

In [36]:
predictor = Predictor()

/home/dafne/anaconda3/envs/filtbubtest/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:431: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


INFO:tensorflow:Restoring parameters from /home/dafne/filter-bubble/e2e-Dutch/data/final/model.max.ckpt


INFO:tensorflow:Restoring parameters from /home/dafne/filter-bubble/e2e-Dutch/data/final/model.max.ckpt


In [37]:
e2e_doc['predicted_clusters'] = predictor.predict(e2e_doc)

In [38]:
e2e_doc['predicted_clusters']

[((0, 0),), ((6, 7), (10, 10))]

Let's print out the tokens together with the clusters

In [39]:
with io.StringIO() as sout:
    e2edutch.conll.output_conll(sout, {'example': e2e_doc['sentences']}, 
                                      {'example': e2e_doc['predicted_clusters']})
    conll_e2e = sout.getvalue()

In [40]:
print(conll_e2e)

#begin document (example);

example	0	Dit	(0)
example	1	is	-
example	2	een	-
example	3	stukje	-
example	4	tekst	-
example	5	.	-

example	0	Deze	(1
example	1	tekst	1)
example	2	dient	-
example	3	als	-
example	4	voorbeeld	(1)
example	5	.	-

#end document

